In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from espn_api.football import League, Team, Player

from collections import Counter

In [20]:
year = 2020

login = pd.read_csv('login.csv')
manager, league_name, league_id, swid, espn_s2 = login.iloc[0]

league = League(league_id=league_id,
                year=year,
                swid=swid, 
                espn_s2=espn_s2)

week = 4
league.load_roster_week(week)

team = league.teams[0]
# player = team.roster[0]

league.load_roster_week(10)
box_scores = league.box_scores(1)
box_score = box_scores[0]

In [343]:
a = [('a', 1), ('b', 5), ('c', 4), ('d', 4), ('e', 2)]
s = [i[1] for i in a]
sorted(s, reverse=True).index(4)

1

In [3]:
# Get a dictionary of the starting roster slots and number of each for the League (Week 1 must have passed already)
starting_roster_slots = Counter([p.slot_position for p in league.box_scores(1)[0].home_lineup if p.slot_position not in ['BE', 'IR']])

In [12]:
def get_top_players(team: Team, slot: str, n: int):
    ''' Takes a list of players and returns a list of the top n players based on points scored. '''
    # Gather players of the desired position
    eligible_players = []
    for player in team.roster:
        if slot in player.eligibleSlots:
            eligible_players.append(player)
            
    return sorted(eligible_players, key=lambda x: x.stats[1]['points'], reverse=True)[:n]


def get_best_lineup(team: Team):
    ''' Returns the best possible lineup for team during the loaded week. '''
    # Save full roster 
    saved_roster = team.roster[:]

    # Find Best Lineup
    best_lineup = []
    for slot in sorted(starting_roster_slots.keys(), key=len):  # Get best RB before best RB/WR/TE
        num_players = starting_roster_slots[slot]
        best_players = get_top_players(team, slot, num_players)
        best_lineup.extend(best_players)
        
        # Remove selected players from consideration for other slots
        for player in best_players:
            team.roster.remove(player)
            
    # Restore original roster
    team.roster = saved_roster

    return np.sum([player.stats[1]['points'] for player in best_lineup])


def get_best_trio(team: Team):
    ''' Returns the the sum of the top QB/RB/Reciever trio for a team during the loaded week. '''
    qb = get_top_players(team, 'QB', 1)[0].stats[1]['points']
    rb = get_top_players(team, 'RB', 1)[0].stats[1]['points']
    wr = get_top_players(team, 'WR', 1)[0].stats[1]['points']
    te = get_top_players(team, 'TE', 1)[0].stats[1]['points']
    best_trio = round(qb + rb + max(wr, te), 2)
    return best_trio

def get_weekly_finish(league: League, team: Team, week: int):
    ''' Returns the rank of a team compared to the rest of the league by points for (for the loaded week) '''
    league_scores = [tm.scores[week-1] for tm in league.teams]
    league_scores = sorted(league_scores, reverse=True)
    return league_scores.index(team.scores[week-1]) + 1

def get_num_out(team: Team):
    ''' Returns the (esimated) number of players who did not play for a team for the loaded week (excluding IR slot players). '''
    num_out = 0
    # TODO: write new code based on if player was injured
    return num_out

def avg_starting_score_slot(lineup: list, slot: str):
    ''' 
    Returns the average score for starting players of a specified slot.
    `lineup` is either BoxScore().away_lineup or BoxScore().home_lineup (a list of BoxPlayers)
    '''
    return np.mean([player.stats[1]['points'] for player in box_score.home_lineup if player.slot_position == slot])

def total_bench_points(lineup: list):
    ''' 
    Returns the total score for bench players
    `lineup` is either BoxScore().away_lineup or BoxScore().home_lineup (a list of BoxPlayers)
    '''
    return np.sum([player.stats[1]['points'] for player in box_score.home_lineup if player.slot_position == 'BE'])
    
    

from tabulate import tabulate as table
def print_weekly_stats(league: league, team: Team, week: int):
    ''' Print the weekly stats for the team during a given week. '''
    # Get the lineup for the team during the specified week
    box_scores = league.box_scores(week)
    for box_score in box_scores:
        if team == box_score.home_team:
            lineup = box_score.home_lineup
        elif team == box_score.away_team:
            lineup = box_score.away_lineup
    
    stats_table = [['Week Score: ', team.scores[week-1]],
                   ['Best Possible Lineup: ', get_best_lineup(team)],
                   ['Opponent Score: ', team.schedule[week-1].scores[week-1]],
                   
                   ['Weekly Finish: ', get_weekly_finish(league, team, week)],
                   ['Best Trio: ', get_best_trio(team)],
                   ['Number of Injuries: ', get_num_out(team)],
                   ['Starting QB pts: ', avg_starting_score_slot(team, 'QB')],
                   ['Avg. Starting RB pts: ', avg_starting_score_slot(team, 'RB')],
                   ['Avg. Starting WR pts: ', avg_starting_score_slot(team, 'WR')],
                   ['Starting TE pts: ', avg_starting_score_slot(team, 'TE')],
                   ['Starting Flex pts: ', avg_starting_score_slot(team, 'RB/WR/TE')],
                   ['Starting DST pts: ', avg_starting_score_slot(team, r'D/ST')],
                   ['Starting K pts: ', avg_starting_score_slot(team, 'K')],
                   ['Total Bench pts: ', total_bench_points(team)]]
    
    print('\n', table(stats_table, headers = ['Week ' + str(week), ''], numalign = 'left'))
    
print_weekly_stats(team, 1)


 Week 1
---------------------  ------
Week Score:            124.68
Best Possible Lineup:  139.58
Opponent Score:        132.74
Weekly Finish:         4
Best Trio:             63.18
Number of Injuries:    0
Starting QB pts:       31.78
Avg. Starting RB pts:  19.1
Avg. Starting WR pts:  5.25
Starting TE pts:       7.3
Starting Flex pts:     7.95
Starting DST pts:      12
Starting K pts:        9
Total Bench pts:       74.96


In [ ]:
#league.power_rankings(week)

In [336]:
def get_weekly_luck_index(league: League, team: Team, week: int, num_teams: int):
    ''' 
    This function returns an index quantifying how 'lucky' a team was in a given week 
    
    Luck index:
        50% probability of playing a team with a lower record
        25% your play compared to previous weeks
        25% opp's play compared to previous weeks
    '''
    opp = team.schedule[week-1]
    
    # Luck Index based on where the team and its opponent finished compared to the rest of the league  
    rank = get_weekly_finish(league, team, week)
    opp_rank = get_weekly_finish(league, opp, week)

    if rank < opp_rank:                                # If the team won...
        luck_index = 5 * (rank - 1) / (num_teams - 2)  # Odds of this team playing a team with a higher score than it
    elif rank > opp_rank:                              # If the team lost or tied...
        luck_index = -5 * (num_teams - rank) / (num_teams - 2)    # Odds of this team playing a team with a lower score than it

    # If the team tied...
    elif rank < (num_teams / 2):                                      
        luck_index = -2.5 * (num_teams - rank - 1) / (num_teams - 2)  # They are only half as unlucky, because tying is not as bad as losing
    else:
        luck_index = 2.5 * (rank - 1) / (num_teams - 2)               # They are only half as lucky, because tying is not as good as winning


    # Update luck index based on how team played compared to normal
    team_score = team.scores[week - 1]
    team_avg = np.mean(team.scores[:week])
    team_std = np.std(team.scores[:week])
    if team_std != 0:
        # Get z-score of the team's performance
        z = (team_score - team_avg) / team_std
        
        # Noramlize the z-score so that a performance 3 std dev's away from the mean has an effect of 2 points on the luck index
        z_norm = z / (3*team_std) * 2.5
        luck_index += z_norm

    # Update luck index based on how opponent played compared to normal
    opp_score = opp.scores[week - 1]
    opp_avg = np.mean(opp.scores[:week])
    opp_std = np.std(opp.scores[:week])
    if team_std != 0:
        # Get z-score of the team's performance
        z = (opp_score - opp_avg) / opp_std
        
        # Noramlize the z-score so that a performance 3 std dev's away from the mean has an effect of 2 points on the luck index
        z_norm = z / (3*opp_std) * 2.5
        luck_index -= z_norm
    
    return luck_index / 10


def get_season_luck_indices(league: League, week: int):
    ''' This function returns an index quantifying how 'lucky' a team was all season long (up to a certain week) '''
    luck_indices = {team:0 for team in league.teams}
    for wk in range(1, week + 1):
        # Update luck_index for each team
        for team in league.teams:
            luck_indices[team] += get_weekly_luck_index(team, week, len(league.teams))
    return luck_indices

In [335]:
# Get luck index
week = 12
sorted(get_season_luck_indices(league, week).items(), key=lambda x: x[1], reverse=True)

[(Team(DJ Chark doo doo doo doo), 5.0068028772958435),
 (Team(Milk and  Dalvin Cookies), 2.9790090731134082),
 (Team(Fresh Prince of Helaire), 2.0835728254697865),
 (Team(Can't Start Mike), -0.01267057171035671),
 (Team(Lockett the OG), -0.08357282546978607),
 (Team(Tannesaurus Rex), -1.0068028772958424),
 (Team(Antonio Brownies ), -2.9790090731134082),
 (Team(Ma Homie), -5.987329428289644)]

In [49]:
week = 5
opp = team.schedule[week-1]



[123.2, 178.7, 146.0, 81.6, 124.58]

In [54]:
get_weekly_luck_index(league.teams[0], 5, 8)

2.124309708013391

In [40]:
3.97 - 1.46

2.5100000000000002

In [11]:
league.box_score

AttributeError: 'League' object has no attribute 'box_score'

In [263]:
# league.box_scores(1)
a = {team:0 for team in league.teams}
a[team] += 10
a

{Team(Fresh Prince of Helaire): 10,
 Team(Lockett the OG): 0,
 Team(Can't Start Mike): 0,
 Team(Milk and  Dalvin Cookies): 0,
 Team(DJ Chark doo doo doo doo): 0,
 Team(Tannesaurus Rex): 0,
 Team(Ma Homie): 0,
 Team(Antonio Brownies ): 0}

[Player(Patriots D/ST)]